# Fine Tuning usando Unsloth library
Código baseado em https://colab.research.google.com/drive/135ced7oHytdxu3N2DNe1Z0kqjyYIkDXp?usp=sharing#scrollTo=2eSvM9zX_2d3

!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

!pip uninstall unsloth -y
!pip uninstall peft -y
!pip uninstall accelerate -y
!pip uninstall bitsandbytes -y
!pip uninstall trl -y
!pip uninstall xformers -y

!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

Configurando ambiente

sudo apt install python3-pip
pip install --upgrade pip
python.exe -m pip install --upgrade pip
pip install notebook
restart
jupyter notebook

pip3 install torch torchvision torchaudio
pip install tensorflow
pip install transformers
pip install git+https://github.com/Keith-Hon/bitsandbytes-windows.git
pip install accelerate
pip install appdirs==1.4.4
pip install datasets
pip install fire==0.5.0
pip install sentencepiece==0.1.97
pip install peft
pip install tensorboardX==2.6
pip install gradio==3.23.0
pip install seaborn
pip install wandb
pip install langchain
pip install langchain-community
pip install langchain_huggingface
pip install chromadb
pip install langchainhub

In [1]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
import re
from llm_utils import salvar_modelo, carregar_modelo, pick_from_dataset, carregar_dataset, alpaca_prompt, print_memory_used_in_training, print_memory_usage
    
RESUME_FROM_CHECKPOINT = False
CONTINUAR_TREINAMENTO_SALVO = True
pasta_base = './Datasets/novo-rel-voto-ementa/'
DATASET_FILE = pasta_base + 'dataset_rel_voto0.json'
BATCH_START = 0
BATCH_END = 0 # 0 means there is no end
BATCH_SIZE = BATCH_END - BATCH_START 

MY_MODEL_NAME = "./fine_tuned_models/relvoto_unsloth_lora_model_llama_31_8b_4bit_pt"
max_seq_length = 10000 #8k roda - 12k estoura

dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

def formatar_paragrafos_sucessivos(texto):
    pattern = '(?<=[0-9"”\)À-Üà-üAEILMORSUXYZaeilmorsuxyz]\.) {0,4}"?(?=[0-9]{1,2} ?[\–\-\.] ?[A-ZÀ-Ü])'
    return re.sub(pattern, '\n\n', texto)
    
# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

# Lista de modelos do site https://github.com/unslothai/unsloth?tab=readme-ov-file em 2024_07_28
fourbit_models_2024_07_28 = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",          # Phi-3 2x faster!d
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

MODEL_NAME = "unsloth/llama-3-8b-bnb-4bit"
MODEL_NAME = fourbit_models[4] # "unsloth/llama-3-70b-bnb-4bit" - não funcionou. faltou memória
MODEL_NAME = fourbit_models_2024_07_28[1]

print('Model name =', MODEL_NAME)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-10-01 10:57:35.127128: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-01 10:57:35.134621: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-01 10:57:35.142505: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-01 10:57:35.144815: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-01 10:57:35.152031: I tensorflow/core/platform/cpu_feature_guar

Model name = unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [2]:
if CONTINUAR_TREINAMENTO_SALVO:
    model_name = MY_MODEL_NAME
    criar_camada_LORA = False
    print('Vamos continuar o treinamento do:', MY_MODEL_NAME)
else:
    model_name = MODEL_NAME
    criar_camada_LORA = True
    
model, tokenizer = carregar_modelo(model_name = model_name, 
                               max_seq_length = max_seq_length, 
                               dtype = dtype, 
                               load_in_4bit = load_in_4bit,
                               criar_camada_LORA = criar_camada_LORA)
print_memory_usage()

Vamos continuar o treinamento do: ./fine_tuned_models/relvoto_unsloth_lora_model_llama_31_8b_4bit_pt
==((====))==  Unsloth: Fast Llama patching release 2024.8
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.988 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Modelo ./fine_tuned_models/relvoto_unsloth_lora_model_llama_31_8b_4bit_pt carregado.
GPU = NVIDIA GeForce RTX 4090. Max memory = 23.988 GB.
6.141 GB of memory reserved.


GPU = NVIDIA GeForce RTX 4090. Max memory = 23.988 GB.
5.77 GB of memory reserved.

# Data Prep

In [3]:
train_dataset, eval_dataset = carregar_dataset(data_files = DATASET_FILE, eos_token = tokenizer.eos_token)
# offset
if BATCH_END != 0:
    train_dataset = pick_from_dataset(train_dataset, 
                                      batch_start = BATCH_START, 
                                      batch_end = BATCH_END)


train_dataset: Dataset({
    features: ['instruction', 'input', 'output', 'desembargador', 'text'],
    num_rows: 9031
})
eval_dataset: Dataset({
    features: ['instruction', 'input', 'output', 'desembargador', 'text'],
    num_rows: 128
})


# Train the model

Now let's use Huggingface TRL's SFTTrainer! More docs here: TRL SFT docs. We do 60 steps to speed things up, but you can set num_train_epochs=1 for a full run, and turn off max_steps=None. We also support TRL's DPOTrainer!

Para determinar o tamanho do treino, posso usar um destes:
- max_steps = BATCH_SIZE
- num_train_epochs=1

In [4]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        num_train_epochs=1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        do_eval = False, #Estou com erro de memória ao converter para FP32 ou FP16 durante evaluate.
        save_steps = 200,
        load_best_model_at_end = False, # Não posso usar evaluate com quantização (memory)
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to="wandb",
    ),
)

Map (num_proc=2):   0%|          | 0/9031 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/128 [00:00<?, ? examples/s]

# Show current memory stats

In [5]:
print_memory_usage()

GPU = NVIDIA GeForce RTX 4090. Max memory = 23.988 GB.
6.141 GB of memory reserved.


# Train

In [7]:
%%time
print(len(train_dataset))
print(len(eval_dataset))
trainer_stats = trainer.train(resume_from_checkpoint = RESUME_FROM_CHECKPOINT)

9031
128


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 9,031 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 8
\        /    Total batch size = 16 | Total steps = 564
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: fabiocollado (fabiocollado-unicamp). Use `wandb login --relogin` to force relogin


Step,Training Loss
10,0.409500
20,0.432200
30,0.425600
40,0.390000
50,0.371200
60,0.412200
70,0.379500
80,0.424900
90,0.458400
100,0.464400


KeyboardInterrupt: 

# Show final memory and time stats

In [ ]:
print_memory_used_in_training(trainer_stats = trainer_stats)

# Saving finetuned models

In [ ]:
salvar_modelo(my_model_name = MY_MODEL_NAME, 
              model = model, 
              tokenizer = tokenizer)
print_memory_usage()

# Loading finetuned models and Eval Dataset

In [ ]:
model, tokenizer = carregar_modelo(model_name = MY_MODEL_NAME, 
                                   max_seq_length = max_seq_length, 
                                   dtype = dtype, 
                                   load_in_4bit = load_in_4bit,
                                   criar_camada_LORA = False,
                                   inference = True)
print_memory_usage()

_, eval_dataset = carregar_dataset(data_files = DATASET_FILE, eos_token = tokenizer.eos_token)

In [ ]:
MY_MODEL_NAME

# Inference
Let's run the model! You can change the instruction and input - leave the output blank!

### For inference, use:
- inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")
- outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
- tokenizer.batch_decode(outputs)

In [ ]:
%%time
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
# Lembre-se que o dataset esta no formato {'instruction': ['Crie uma ementa para o seguinte voto:',  
# 'Crie uma ementa para o seguinte voto:'], ...,  'input': [...], 'output': [...], 'text':[...]}

def remove_list_of_words(text, list_of_words):
    for word in list_of_words:
        text = text.replace(word,'')
    return text
def inference_test(model, tokenizer, dataset, start, end, alpaca_prompt, tamanho_maximo_da_ementa):
    sub_list = dataset[start:end]
    for instruction, input, output in zip(sub_list['instruction'], sub_list['input'], sub_list['output']):
        print('-----------------------------------------INPUT-----------------------------------------------------')
        print(input)
        print('-------------------------------------DESIRED-OUTPUT------------------------------------------------')
        print(formatar_paragrafos_sucessivos(output))
        print('------------------------------------GENERATED-OUTPUT-----------------------------------------------')
        prompt = tokenizer([alpaca_prompt.format(instruction, input, '')], return_tensors = "pt").to("cuda")
        generated_output = model.generate(**prompt, max_new_tokens = tamanho_maximo_da_ementa, use_cache = True)
        generated_output = tokenizer.batch_decode(generated_output)
        texto = remove_list_of_words(generated_output[0], ['<|end_of_text|>', '</s>', '<|eot_id|>'])
        if 'Response:\n' in generated_output[0]:
            response_word = 'Response:\n'
        elif 'Resposta:\n' in generated_output[0]:
            response_word = 'Resposta:\n'
        else:
            raise Exception("Resposta não encontrada no texto gerado. Ou formato inválido do texto gerado.")
        print(formatar_paragrafos_sucessivos(texto.split(response_word)[1]))
        print('---------------------------------------------------------------------------------------------------')
        print()

inference_test(model = model,
               dataset = eval_dataset, start = 0, end = 127,
               tokenizer = tokenizer,
               alpaca_prompt = alpaca_prompt,
               tamanho_maximo_da_ementa = 10000)

You can also use a TextStreamer for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# alpaca_prompt = Copied from above

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)